In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero
dir_ = '../../../../../data'
group_dir_ = os.path.join(dir_, 'groups/random')

In [2]:
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'
df = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

num_user = len(svd['uid'].unique())
num_user

953

In [3]:
pop_dict = {}
for i in tqdm(df['tid'].unique()):
    pop_dict[i] = len(df[df['tid']==i])/num_user

In [4]:
svd['count'] = 1
svd[:5]

,uid,tid,rating,count
0,0,9,2.243909,1
1,0,15,2.160610,1
2,0,22,2.145235,1
3,0,28,2.204445,1
4,0,33,2.166677,1


In [5]:
# Count POP to DF
tid_list = []
pop_list = []
for i in df['tid'].unique():
    tid_list.append(i)
    pop_list.append(len(df[df['tid']==i])/num_user*5)
    
d = {'tid': tid_list, 'rating': pop_list}
df_pop = pd.DataFrame(data=d)
df_pop = df_pop.sort_values(by=['rating'], ascending=False)
df_pop[:10]

,tid,rating
70,70,4.034627
5521,5521,2.911857
390,390,2.822665
2519,2519,2.801679
83,83,2.796432
13496,13496,2.785939
210,210,2.686254
5716,5716,2.681007
1036,1036,2.544596
1464,1464,2.528856


In [6]:
lambdas = []
for i in range(11):
    lambdas.append(i/10)
lambdas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [7]:
top_n_size = 20

In [8]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups = groups[int(len(groups)/5*0):int(len(groups)/5*1)]
    groups_n.append(groups)

NOVs = []
COVs = []
HitNOVs = []
HitCOVs = []
for lambda_ in tqdm(lambdas):
    top_n_lists_g = []
    top_n_items_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []
        top_n_items = pd.DataFrame()

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
            
            if len(top_n_items) == 0:
                top_n_items = rating_table
            else:
                top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()
                
        top_n_lists_g.append(top_n_lists)   
        top_n_items_g.append(top_n_items)
        
    COV = []    
    NOV = []
    HitNOV = []
    HitCOV = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]
        top_n_items = top_n_items_g[i]

        
        nov = 0
        hitNov = 0
        hits = np.zeros(len(df['tid'].unique()), dtype=bool)
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    nov += (1-pop_dict[tid])
                    if len(t) > 0:
                        hits[int(tid)] = True
                        hitNov += (1-pop_dict[tid])
        NOV.append(nov/top_n_size/(len(groups)*len(groups[0])))
        COV.append(len(top_n_items)/len(df['tid'].unique()))
        HitNOV.append(hitNov/top_n_size/(len(groups)*len(groups[0])))
        HitCOV.append(list(hits).count(True)/len(df['tid'].unique()))
    NOVs.append(NOV)
    COVs.append(COV)
    HitNOVs.append(HitNOV)
    HitCOVs.append(HitCOV)
    
# for NOV in NOVs:
#     print(NOV)
# print()
for HitNOV in HitNOVs:
    print(HitNOV)
print()  
# for COV in COVs:
#     print(COV)
print()    
for HitCOV in HitCOVs:
    print(HitCOV)


[0.048334345833103254, 0.050532154099835205, 0.05104568998236259, 0.05260368918097982, 0.04962568685193341, 0.04390257443772662, 0.045992917103882515, 0.04995586202301844, 0.04421577290550618]
[0.05778124482244439, 0.056799191636547466, 0.05837305485476999, 0.05568509416247868, 0.053913504609100944, 0.04695114842019359, 0.046743407545964705, 0.052000088831148705, 0.045520516927155245]
[0.06758435963991825, 0.06418439125679427, 0.06460784532607004, 0.06306097089523394, 0.059427783231222366, 0.05091551602569444, 0.050138008120808444, 0.05545811888938866, 0.04818882200254045]
[0.07803501408295137, 0.07448408534452615, 0.06997946015940708, 0.06930413652178719, 0.06265725665414261, 0.056855266299127794, 0.05337749213011543, 0.05949216342710581, 0.050194952228419944]
[0.08985061026122493, 0.08096903679275148, 0.08205526779933472, 0.07942784558458059, 0.0694566676821357, 0.06228756863594222, 0.05835405356083763, 0.061401200330896154, 0.054847848898216166]
[0.09762467553984644, 0.088383384133

In [9]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups = groups[int(len(groups)/5*1):int(len(groups)/5*2)]
    groups_n.append(groups)

NOVs = []
COVs = []
HitNOVs = []
HitCOVs = []
for lambda_ in tqdm(lambdas):
    top_n_lists_g = []
    top_n_items_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []
        top_n_items = pd.DataFrame()

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
            
            if len(top_n_items) == 0:
                top_n_items = rating_table
            else:
                top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()
                
        top_n_lists_g.append(top_n_lists)   
        top_n_items_g.append(top_n_items)
        
    COV = []    
    NOV = []
    HitNOV = []
    HitCOV = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]
        top_n_items = top_n_items_g[i]

        
        nov = 0
        hitNov = 0
        hits = np.zeros(len(df['tid'].unique()), dtype=bool)
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    nov += (1-pop_dict[tid])
                    if len(t) > 0:
                        hits[int(tid)] = True
                        hitNov += (1-pop_dict[tid])
        NOV.append(nov/top_n_size/(len(groups)*len(groups[0])))
        COV.append(len(top_n_items)/len(df['tid'].unique()))
        HitNOV.append(hitNov/top_n_size/(len(groups)*len(groups[0])))
        HitCOV.append(list(hits).count(True)/len(df['tid'].unique()))
    NOVs.append(NOV)
    COVs.append(COV)
    HitNOVs.append(HitNOV)
    HitCOVs.append(HitCOV)
    
# for NOV in NOVs:
#     print(NOV)
# print()
for HitNOV in HitNOVs:
    print(HitNOV)
print()  
# for COV in COVs:
#     print(COV)
print()    
for HitCOV in HitCOVs:
    print(HitCOV)


[0.04868062075440444, 0.05040834568641495, 0.04757044639734176, 0.04807726293698577, 0.048159594700944434, 0.04970435883342499, 0.05103975384749918, 0.0499353198198949, 0.04838984922957977]
[0.058273319710609214, 0.057438498309432234, 0.052582579136061625, 0.04884575026232954, 0.051614965897166874, 0.054794661247966635, 0.05376005596362374, 0.051833808024783934, 0.051285138344286794]
[0.06911194565637596, 0.06536945429915007, 0.05712497813920958, 0.05457944441376268, 0.05598302509618753, 0.058559436366361906, 0.05737829004896824, 0.0541289828278286, 0.05429833765946875]
[0.08345860716849846, 0.07592648112060504, 0.06440462137110882, 0.061491412161042674, 0.059334557537600585, 0.06206188199892295, 0.05979855281566986, 0.057393805137771536, 0.059682995526591946]
[0.0936129673606893, 0.08588778405147775, 0.07160665879678217, 0.06439885127298833, 0.06657594438614903, 0.06690012602919214, 0.061957579136061584, 0.061322362686475965, 0.06420030927265703]
[0.09915005246589723, 0.0939661442284

In [10]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups = groups[int(len(groups)/5*2):int(len(groups)/5*3)]
    groups_n.append(groups)

NOVs = []
COVs = []
HitNOVs = []
HitCOVs = []
for lambda_ in tqdm(lambdas):
    top_n_lists_g = []
    top_n_items_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []
        top_n_items = pd.DataFrame()

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
            
            if len(top_n_items) == 0:
                top_n_items = rating_table
            else:
                top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()
                
        top_n_lists_g.append(top_n_lists)   
        top_n_items_g.append(top_n_items)
        
    COV = []    
    NOV = []
    HitNOV = []
    HitCOV = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]
        top_n_items = top_n_items_g[i]

        
        nov = 0
        hitNov = 0
        hits = np.zeros(len(df['tid'].unique()), dtype=bool)
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    nov += (1-pop_dict[tid])
                    if len(t) > 0:
                        hits[int(tid)] = True
                        hitNov += (1-pop_dict[tid])
        NOV.append(nov/top_n_size/(len(groups)*len(groups[0])))
        COV.append(len(top_n_items)/len(df['tid'].unique()))
        HitNOV.append(hitNov/top_n_size/(len(groups)*len(groups[0])))
        HitCOV.append(list(hits).count(True)/len(df['tid'].unique()))
    NOVs.append(NOV)
    COVs.append(COV)
    HitNOVs.append(HitNOV)
    HitCOVs.append(HitCOV)
    
# for NOV in NOVs:
#     print(NOV)
# print()
for HitNOV in HitNOVs:
    print(HitNOV)
print()  
# for COV in COVs:
#     print(COV)
print()    
for HitCOV in HitCOVs:
    print(HitCOV)


[0.04660821781631419, 0.04941139821615953, 0.04826499743251996, 0.04330203788590051, 0.049547834230330975, 0.049634404303869194, 0.04743791535501931, 0.04898343854272507, 0.052402938090241394]
[0.05574943392058328, 0.056157911420077, 0.05326349043334601, 0.04735903241840175, 0.0520763519841136, 0.052417317632427844, 0.05074053427771955, 0.049595818273677726, 0.054164687689843755]
[0.06635472469210807, 0.06475985921650931, 0.05781602330825391, 0.05023747721875522, 0.05532472441300257, 0.05504200047746744, 0.0549123928821267, 0.05353853328669704, 0.05559866350030381]
[0.0792654774396643, 0.07298033621895773, 0.06468989305887342, 0.05459739327331976, 0.06099104130702148, 0.05959598483208134, 0.05913972324239248, 0.05660626148558996, 0.05787264593803505]
[0.09164825757994152, 0.08239933105981118, 0.07262563907927928, 0.06091925774562329, 0.06730246307641971, 0.06752694082179916, 0.06593159758656876, 0.060042916548687826, 0.060614403269453825]
[0.0997244159717237, 0.08996453086743621, 0.08

In [11]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups = groups[int(len(groups)/5*3):int(len(groups)/5*4)]
    groups_n.append(groups)

NOVs = []
COVs = []
HitNOVs = []
HitCOVs = []
for lambda_ in tqdm(lambdas):
    top_n_lists_g = []
    top_n_items_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []
        top_n_items = pd.DataFrame()

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
            
            if len(top_n_items) == 0:
                top_n_items = rating_table
            else:
                top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()
                
        top_n_lists_g.append(top_n_lists)   
        top_n_items_g.append(top_n_items)
        
    COV = []    
    NOV = []
    HitNOV = []
    HitCOV = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]
        top_n_items = top_n_items_g[i]

        
        nov = 0
        hitNov = 0
        hits = np.zeros(len(df['tid'].unique()), dtype=bool)
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    nov += (1-pop_dict[tid])
                    if len(t) > 0:
                        hits[int(tid)] = True
                        hitNov += (1-pop_dict[tid])
        NOV.append(nov/top_n_size/(len(groups)*len(groups[0])))
        COV.append(len(top_n_items)/len(df['tid'].unique()))
        HitNOV.append(hitNov/top_n_size/(len(groups)*len(groups[0])))
        HitCOV.append(list(hits).count(True)/len(df['tid'].unique()))
    NOVs.append(NOV)
    COVs.append(COV)
    HitNOVs.append(HitNOV)
    HitCOVs.append(HitCOV)
    
# for NOV in NOVs:
#     print(NOV)
# print()
for HitNOV in HitNOVs:
    print(HitNOV)
print()  
# for COV in COVs:
#     print(COV)
print()    
for HitCOV in HitCOVs:
    print(HitCOV)


[0.052666924393880814, 0.04525058711837308, 0.04950813221406092, 0.051459932622742684, 0.04934280998600917, 0.051775512583487414, 0.04860883394543553, 0.044084678292443294, 0.04634892582978964]
[0.06339813331860614, 0.05078837644420016, 0.052258219657222875, 0.05409537747832337, 0.053368201294158875, 0.05641638490536709, 0.05078261629940545, 0.047855838149647216, 0.048671232120174567]
[0.0760736179378142, 0.05837649971962673, 0.05927034146554742, 0.05868172529960795, 0.05905309986009103, 0.05933198976221014, 0.05366332196572235, 0.0518510190598334, 0.050358977191141595]
[0.08892776274369035, 0.06706002209674826, 0.06516346406086047, 0.06435908764566198, 0.06443713929695709, 0.06472987002892565, 0.05765510230849951, 0.055102239100140525, 0.052974540233059086]
[0.09653283260617455, 0.07587817918353074, 0.07485353270374258, 0.07155105760203238, 0.07036605893669128, 0.07074318359732841, 0.0627811844176286, 0.05872460678336867, 0.05736179378141056]
[0.10228033357265155, 0.08217686281694676

In [12]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups = groups[int(len(groups)/5*4):int(len(groups)/5*5)]
    groups_n.append(groups)

NOVs = []
COVs = []
HitNOVs = []
HitCOVs = []
for lambda_ in tqdm(lambdas):
    top_n_lists_g = []
    top_n_items_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []
        top_n_items = pd.DataFrame()

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
            
            if len(top_n_items) == 0:
                top_n_items = rating_table
            else:
                top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()
                
        top_n_lists_g.append(top_n_lists)   
        top_n_items_g.append(top_n_items)
        
    COV = []    
    NOV = []
    HitNOV = []
    HitCOV = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]
        top_n_items = top_n_items_g[i]

        
        nov = 0
        hitNov = 0
        hits = np.zeros(len(df['tid'].unique()), dtype=bool)
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    nov += (1-pop_dict[tid])
                    if len(t) > 0:
                        hits[int(tid)] = True
                        hitNov += (1-pop_dict[tid])
        NOV.append(nov/top_n_size/(len(groups)*len(groups[0])))
        COV.append(len(top_n_items)/len(df['tid'].unique()))
        HitNOV.append(hitNov/top_n_size/(len(groups)*len(groups[0])))
        HitCOV.append(list(hits).count(True)/len(df['tid'].unique()))
    NOVs.append(NOV)
    COVs.append(COV)
    HitNOVs.append(HitNOV)
    HitCOVs.append(HitCOV)
    
# for NOV in NOVs:
#     print(NOV)
# print()
for HitNOV in HitNOVs:
    print(HitNOV)
print()  
# for COV in COVs:
#     print(COV)
print()    
for HitCOV in HitCOVs:
    print(HitCOV)


[0.047424798880727585, 0.04819402544596021, 0.047622639034627545, 0.048548627602584674, 0.04781146161245194, 0.04876758678287692, 0.05075665661070312, 0.051649483391351246, 0.051971613188269775]
[0.057321725253585254, 0.05480636804826874, 0.05309220881427077, 0.05319600154636335, 0.05042847149352924, 0.051855311904405066, 0.05375513728576436, 0.05412842763314962, 0.056163638371900436]
[0.06919268100734527, 0.0621417562959077, 0.05952638597411686, 0.05751394488319441, 0.055255334032878654, 0.054829004004539966, 0.054741058936691234, 0.056322279407274224, 0.05947699784613691]
[0.07796568948933197, 0.07133531173487238, 0.06610211175236101, 0.06391865024576136, 0.06075796825813226, 0.058140512238473635, 0.05792043765302559, 0.0612449130287536, 0.06173993483183301]
[0.0866957961699895, 0.08381918940188875, 0.07440565975865682, 0.07156099850886397, 0.06719460475690804, 0.061221277598132694, 0.06290907004197278, 0.06686875752982789, 0.06655740873695255]
[0.09627601434067855, 0.09249956278419